<a href="https://colab.research.google.com/github/XXV007/Smart-Resume-Ranker/blob/main/Smart_Resume_Ranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer.six python-docx scikit-learn pandas


In [ ]:
import os
import re
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pdfminer.high_level import extract_text
from docx import Document
from google.colab import files

In [ ]:
def extract_text_from_pdf(path: str) -> str:
    try:
        return extract_text(path)
    except Exception as e:
        print(f"⚠️ PDF parse error ({path}): {e}")
        return ""

def extract_text_from_docx(path: str) -> str:
    try:
        doc = Document(path)
        return "\n".join([p.text for p in doc.paragraphs])
    except Exception as e:
        print(f"⚠️ DOCX parse error ({path}): {e}")
        return ""

def extract_resume_text(path: str) -> str:
    if path.lower().endswith(".pdf"):
        return extract_text_from_pdf(path)
    elif path.lower().endswith(".docx"):
        return extract_text_from_docx(path)
    else:
        print(f"⚠️ Unsupported filetype: {path}")
        return ""

In [ ]:
# This will pop up a file picker — select your PDF/DOCX resumes
uploaded = files.upload()

resume_texts = {}
for fname in uploaded.keys():
    text = extract_resume_text(fname)
    resume_texts[fname] = text
print(f"Parsed {len(resume_texts)} resumes.")

Saving Resume constant update.docx to Resume constant update (1).docx
Parsed 1 resumes.


In [ ]:
job_description = """
We are looking for a Machine Learning Engineer with:
- Strong NLP experience (spaCy, NLTK)
- Python & Scikit-Learn expertise
- Familiarity with TensorFlow or PyTorch
- Proven ability to deploy models in production
"""

In [ ]:
# Build TF-IDF matrix (JD + all resumes)
docs = [job_description] + list(resume_texts.values())
vectorizer = TfidfVectorizer(stop_words="english")
tfidf = vectorizer.fit_transform(docs)

# Cosine similarity between JD (0) and each resume (1…N)
scores = cosine_similarity(tfidf[0:1], tfidf[1:]).flatten()

In [ ]:
ranking = pd.DataFrame({
    "Filename": list(resume_texts.keys()),
    "Score": scores
}).sort_values("Score", ascending=False).reset_index(drop=True)

ranking.style.format({"Score": "{:.3f}"})

,Filename,Score
0,Resume constant update (1).docx,0.047


In [31]:
threshold = 0.1
filtered = ranking[ranking.Score >= threshold]
display(filtered)

,Filename,Score
